In [1]:
import pandas as pd
import numpy as np
from sklearn.preprocessing import StandardScaler
from sklearn.decomposition import PCA
import seaborn as sns
import matplotlib.pyplot as plt
import optuna
# import lightgbm as lgb
from sklearn.model_selection import train_test_split
from sklearn.metrics import roc_auc_score
#plt.style.use('fivethirtyeight')
import xgboost as xgb
import sklearn
import random
from NecessaryModules.getData import getData

c:\Users\USER\AppData\Local\Programs\Python\Python311\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
import warnings
warnings.filterwarnings("ignore")

In [3]:
import xgboost as xgb
from sklearn.metrics import mean_absolute_error

In [4]:
filename = '../shot_listed_dataset/CoverTypeNewNormalized.csv'
dataset=getData(filename).values.tolist()
random.shuffle(dataset)
dataset=pd.DataFrame(dataset) #from dataset_list to dataset_dataframe after suffling
# print(dataset)

   Elevation    Aspect     Slope  Horizontal_Distance_To_Hydrology  \
0   0.850622  0.105556  0.150943                          0.045351   
1   0.879149  0.163889  0.188679                          0.426304   
2   0.767116  0.327778  0.320755                          0.068027   
3   0.791494  0.338889  0.226415                          0.294785   
4   0.867220  0.575000  0.584906                          0.385488   

   Vertical_Distance_To_Hydrology  Horizontal_Distance_To_Roadways  \
0                       -0.003509                         0.631060   
1                        0.189474                         0.177358   
2                        0.038596                         0.259652   
3                       -0.077193                         0.085486   
4                        0.314035                         0.074020   

   Hillshade_9am  Hillshade_Noon  Hillshade_3pm  \
0       0.866142        0.874016          0.548   
1       0.897638        0.866142          0.500   
2    

In [5]:
def split_data(data):
    # splitting the data
    
    msk = np.random.rand(len(data)) < 0.8
    yy = [54] #test title is 54 after suffleing the dataframe 
    train = data[msk]
    test = data[~msk]
    x_train = train[train.columns[:train.shape[1] - 1]]  # all columns except the last are x variables
    y_train = train[yy[0]].tolist()  # the last column as y variable
    x_test = test[test.columns[:test.shape[1] - 1]]
    y_test = test[yy[0]].tolist()  # the last column as y variable
    y_train = [0 if i == 7 else i for i in y_train]  # replace label 7 with 0
    y_test = [0 if i == 7 else i for i in y_test]  # replace label 7 with 0
    return x_train, y_train, x_test, y_test

In [6]:
X=dataset.iloc[:,0:-1]
y=dataset.iloc[:,-1]

In [7]:
pcaTrain = PCA(n_components=40).fit(X)# --train-- pca with the "dataset" Dataframe
print(pcaTrain)
compressed_dataset_X= pcaTrain.transform(X) # --transform-- the original "dataset" Dataframe into a lower-dimensional space based on the model's learned parameters
print(type(compressed_dataset_X))
print(len(compressed_dataset_X))
print(compressed_dataset_X)

PCA(n_components=40)
<class 'numpy.ndarray'>
19229
[[ 9.19733798e-01 -2.83825094e-01 -7.67803395e-01 ... -2.64719341e-03
  -7.64484831e-04 -9.41555779e-04]
 [-6.41619257e-03  9.54155266e-01 -2.72383198e-02 ... -3.00913770e-02
   4.21891326e-02 -1.46033684e-02]
 [-5.07633296e-02  8.96393552e-01  5.37173569e-01 ... -1.78661529e-02
   5.31383713e-02 -4.63994023e-03]
 ...
 [ 7.94161411e-01 -2.58908077e-01 -2.41996054e-01 ... -1.13335427e-02
   1.77906692e-02 -6.98357587e-03]
 [-6.76464369e-01 -2.67087519e-01 -3.15591232e-01 ... -3.41453353e-02
   1.60450851e-02 -1.08531527e-02]
 [-5.94956148e-01 -2.67593643e-01 -3.12702999e-01 ... -1.30569757e-02
   2.94042004e-02  8.55628273e-03]]


In [8]:
# from NecessaryModules.splitData import split_data
X_train, y_train, X_test, y_test = split_data(dataset)
print(len(X_train))

15270


In [9]:
compressed_dataset_X_train= pcaTrain.transform(X_train) # --transform-- the original "dataset" Dataframe into a lower-dimensional space based on the model's learned parameters
print(type(compressed_dataset_X_train))
print(len(compressed_dataset_X_train))
print(compressed_dataset_X_train)

compressed_dataset_X_test= pcaTrain.transform(X_test) # --transform-- the original "dataset" Dataframe into a lower-dimensional space based on the model's learned parameters
print(type(compressed_dataset_X_test))
print(len(compressed_dataset_X_test))
print(compressed_dataset_X_test)

<class 'numpy.ndarray'>
15270
[[-6.41619257e-03  9.54155266e-01 -2.72383198e-02 ... -3.00913770e-02
   4.21891326e-02 -1.46033684e-02]
 [-5.07633296e-02  8.96393552e-01  5.37173569e-01 ... -1.78661529e-02
   5.31383713e-02 -4.63994023e-03]
 [ 1.07607793e-01  9.44639545e-01 -9.34689242e-02 ... -1.17492582e-02
   1.33037033e-03  6.42158168e-05]
 ...
 [ 1.00973870e+00 -6.10951789e-01  8.37897656e-01 ... -2.37449598e-02
   3.55994617e-02 -1.11354803e-02]
 [-7.34970340e-01 -3.39148342e-01  1.55885826e-01 ...  3.17198283e-02
  -2.60055053e-02  2.64607614e-02]
 [-5.94956148e-01 -2.67593643e-01 -3.12702999e-01 ... -1.30569757e-02
   2.94042004e-02  8.55628273e-03]]
<class 'numpy.ndarray'>
3959
[[ 9.19733798e-01 -2.83825094e-01 -7.67803395e-01 ... -2.64719341e-03
  -7.64484831e-04 -9.41555779e-04]
 [-2.59112913e-02  1.32182782e-01  1.30957929e-01 ...  9.15979521e-04
  -1.56376557e-03 -6.36055051e-04]
 [ 8.72644441e-01 -2.61906719e-01 -6.69431764e-01 ...  3.03102279e-02
  -1.46845133e-02  1.2751

# tune XG-Boost

In [10]:
# train the model with random parameter

# Preparing the regressor and fitting data with some parameters
xg_reg = xgb.XGBRegressor(objective ='reg:linear', 
                          colsample_bytree = 1, 
                          learning_rate = 0.3,
                          max_depth = 15, 
                          alpha = 10, 
                          n_estimators = 100, 
                          verbose=1, 
                          min_child_weight = 1, 
                          colsample_bylevel = 1, 
                          reg_alpha = 2) 

xg_reg.fit(compressed_dataset_X_train, y_train)

XGBRegressor(alpha=10, base_score=None, booster=None, callbacks=None,
             colsample_bylevel=1, colsample_bynode=None, colsample_bytree=1,
             device=None, early_stopping_rounds=None, enable_categorical=False,
             eval_metric=None, feature_types=None, gamma=None, grow_policy=None,
             importance_type=None, interaction_constraints=None,
             learning_rate=0.3, max_bin=None, max_cat_threshold=None,
             max_cat_to_onehot=None, max_delta_step=None, max_depth=15,
             max_leaves=None, min_child_weight=1, missing=nan,
             monotone_constraints=None, multi_strategy=None, n_estimators=100,
             n_jobs=None, num_parallel_tree=None, ...)

In [11]:
# Finding mean absolute error
preds = xg_reg.predict(compressed_dataset_X_test)
preds = preds.astype(int)
mae = np.abs(y_test-preds).mean()
mae

0.6350088406163172

In [12]:
# Defining the best parameter with optuna
from sklearn.model_selection import KFold
K = 5 # 5 fold cross validation
kf = KFold(n_splits = K, random_state = 42, shuffle = True)


train_tunned = np.array(compressed_dataset_X)
target_train_tunned = np.array(y)

# test = np.array(X)
# target_train = df_train['congestion'].values

# test_preds = np.zeros((len(test)))
oof_preds = np.zeros((len(train_tunned)))
abs_error_arr=[]
def objective(trial):

 for train_index, val_index in (kf.split(compressed_dataset_X)):
    train_X, valid_X = train_tunned[train_index], train_tunned[val_index]
    train_Y, valid_Y = target_train_tunned[train_index], target_train_tunned[val_index]
    
        
        # train_x, test_x, train_y, test_y = train_test_split(data, target, test_size=0.25,random_state=0)
    param = {
            # 'tree_method':'gpu_hist',  # this parameter means using the GPU when training our model to speedup the training process 
            'lambda': trial.suggest_loguniform('lambda', 1e-3, 10.0),
            'alpha': trial.suggest_loguniform('alpha', 1e-3, 10.0),
            'colsample_bytree': trial.suggest_categorical('colsample_bytree', [0.3,0.4,0.5,0.6,0.7,0.8,0.9, 1.0]),
            'colsample_bylevel': trial.suggest_categorical('colsample_bylevel', [0.3,0.4,0.5,0.6,0.7,0.8,0.9, 1.0]),
            'subsample': trial.suggest_categorical('subsample', [0.4,0.5,0.6,0.7,0.8,1.0]),
            'learning_rate': trial.suggest_categorical('learning_rate', [1e-5,1e-4,1e-3,0.001,0.002,0.003,0.004,0.005,0.006,0.007,0.008,0.009,0.01,0.04,0.08,0.12,0.20, 0.24, 0.30, 0.38, 0.42, 0.48, 0.50, 0.60, 0.70, 0.80, 0.90, 1.0]),
            'n_estimators': trial.suggest_int('n_estimators', 1, 300),
            'max_depth': trial.suggest_categorical('max_depth', [5,7,9,11,13,15,17,20,25,30,35,40]),
            'random_state': trial.suggest_categorical('random_state', [24,48,2020]),
            'min_child_weight': trial.suggest_int('min_child_weight', 1, 300),
        }
    model = xgb.XGBRegressor(**param)  
        
    model.fit(train_X,train_Y,eval_set=[(valid_X,valid_Y)],early_stopping_rounds=100,verbose=False)
        
    preds = model.predict(valid_X)
    oof_preds[val_index] = preds # preds gives total 3846 predicted value for valid_X


 
 print(len(oof_preds))
      
 mae = mean_absolute_error(target_train_tunned, oof_preds)
        
 return mae

In [13]:
# this is the main optuna for optimizing a object-->"objective"
study = optuna.create_study(direction='minimize',study_name="XG_tune")

[I 2024-02-18 23:17:33,364] A new study created in memory with name: XG_tune


In [14]:

study.optimize(objective, n_trials=100) # 150 bar "objective" object ke trial kore kore dekhbe, here total 5*100=750
print('Number of finished trials:', len(study.trials))
print('Best trial:', study.best_trial.params)

[I 2024-02-18 23:17:56,064] Trial 0 finished with value: 1.2916181286744328 and parameters: {'lambda': 0.0035981646839084885, 'alpha': 1.3699259103358323, 'colsample_bytree': 0.9, 'colsample_bylevel': 0.7, 'subsample': 0.4, 'learning_rate': 0.008, 'n_estimators': 86, 'max_depth': 30, 'random_state': 24, 'min_child_weight': 21}. Best is trial 0 with value: 1.2916181286744328.


19229


[I 2024-02-18 23:18:07,696] Trial 1 finished with value: 0.8510451608943507 and parameters: {'lambda': 1.0819728208304342, 'alpha': 0.001331565007652033, 'colsample_bytree': 0.9, 'colsample_bylevel': 1.0, 'subsample': 0.4, 'learning_rate': 0.42, 'n_estimators': 210, 'max_depth': 7, 'random_state': 24, 'min_child_weight': 135}. Best is trial 1 with value: 0.8510451608943507.


19229


[I 2024-02-18 23:18:14,541] Trial 2 finished with value: 1.0688850202279132 and parameters: {'lambda': 0.04603107232455046, 'alpha': 0.017260660697964662, 'colsample_bytree': 0.8, 'colsample_bylevel': 0.9, 'subsample': 0.4, 'learning_rate': 0.9, 'n_estimators': 153, 'max_depth': 7, 'random_state': 24, 'min_child_weight': 239}. Best is trial 1 with value: 0.8510451608943507.


...

19229


[I 2024-02-19 00:09:46,926] Trial 98 finished with value: 1.5592786754198693 and parameters: {'lambda': 6.608966485358635, 'alpha': 0.6607467260336223, 'colsample_bytree': 1.0, 'colsample_bylevel': 0.8, 'subsample': 0.7, 'learning_rate': 0.001, 'n_estimators': 206, 'max_depth': 30, 'random_state': 2020, 'min_child_weight': 26}. Best is trial 95 with value: 0.5728990322120254.


19229


[I 2024-02-19 00:11:28,038] Trial 99 finished with value: 0.5716122463812666 and parameters: {'lambda': 4.154552497796218, 'alpha': 0.43516569590594656, 'colsample_bytree': 1.0, 'colsample_bylevel': 0.8, 'subsample': 0.7, 'learning_rate': 0.08, 'n_estimators': 253, 'max_depth': 17, 'random_state': 2020, 'min_child_weight': 16}. Best is trial 99 with value: 0.5716122463812666.


19229
Number of finished trials: 100
Best trial: {'lambda': 4.154552497796218, 'alpha': 0.43516569590594656, 'colsample_bytree': 1.0, 'colsample_bylevel': 0.8, 'subsample': 0.7, 'learning_rate': 0.08, 'n_estimators': 253, 'max_depth': 17, 'random_state': 2020, 'min_child_weight': 16}


In [15]:
import pickle
model_tuned=xgb.XGBRegressor(**study.best_trial.params)
pickle.dump(model_tuned, open("all_pkl_file/all_tuned_model/PLC_XGB.pkl", "wb"))

In [16]:
from matplotlib import pyplot as plt
fig_study = optuna.visualization.plot_parallel_coordinate(study, params=["lambda", "alpha","colsample_bytree","colsample_bylevel","subsample","learning_rate","n_estimators","max_depth","random_state","min_child_weight"])
fig_study.show()

In [17]:
#evaluate the trial
print("Number of finished trials: ", len(study.trials))
print("Best trial:")
trial = study.best_trial
print("  Value: ", trial.value)
print("  Params: ")
for key, value in trial.params.items():
    print("    {}: {}".format(key, value))

Number of finished trials:  100
Best trial:
  Value:  0.5716122463812666
  Params: 
    lambda: 4.154552497796218
    alpha: 0.43516569590594656
    colsample_bytree: 1.0
    colsample_bylevel: 0.8
    subsample: 0.7
    learning_rate: 0.08
    n_estimators: 253
    max_depth: 17
    random_state: 2020
    min_child_weight: 16


# process the whole dataset

In [18]:
import pickle

# Load the saved pickle file
with open('all_pkl_file/all_tuned_model/PLC_XGB.pkl', 'rb') as f:
    tunned_model = pickle.load(f)

In [19]:
tunned_model

XGBRegressor(alpha=0.43516569590594656, base_score=None, booster=None,
             callbacks=None, colsample_bylevel=0.8, colsample_bynode=None,
             colsample_bytree=1.0, device=None, early_stopping_rounds=None,
             enable_categorical=False, eval_metric=None, feature_types=None,
             gamma=None, grow_policy=None, importance_type=None,
             interaction_constraints=None, lambda=4.154552497796218,
             learning_rate=0.08, max_bin=None, max_cat_threshold=None,
             max_cat_to_onehot=None, max_delta_step=None, max_depth=17,
             max_leaves=None, min_child_weight=16, missing=nan,
             monotone_constraints=None, multi_strategy=None, n_estimators=253,
             n_jobs=None, ...)

In [20]:
filename_2 = '../shot_listed_dataset/Cover_type_whole_genuine_dataset_normalization.csv'
dataset_2=getData(filename_2).values.tolist()
random.shuffle(dataset_2)

dataset_2=pd.DataFrame(dataset_2) #from dataset_list to dataset_dataframe after suffling
print(dataset_2)

   Elevation    Aspect     Slope  Horizontal_Distance_To_Hydrology  \
0   0.672888  0.141667  0.045455                          0.184681   
1   0.671332  0.155556  0.030303                          0.151754   
2   0.726801  0.386111  0.136364                          0.191840   
3   0.721877  0.430556  0.272727                          0.173228   
4   0.672628  0.125000  0.030303                          0.109520   

   Vertical_Distance_To_Hydrology  Horizontal_Distance_To_Roadways  \
0                        0.000000                         0.071659   
1                       -0.009983                         0.054798   
2                        0.108153                         0.446817   
3                        0.196339                         0.434172   
4                       -0.001664                         0.054939   

   Hillshade_9am  Hillshade_Noon  Hillshade_3pm  \
0       0.870079        0.913386       0.582677   
1       0.866142        0.925197       0.594488   
2    

In [21]:
# checking for NaN value
result = dataset_2.columns[dataset_2.isna().any()].tolist()
result

[]

In [22]:
X_2=dataset_2.iloc[:,0:-1]
y_2=dataset_2.iloc[:,-1]
X_train_2, y_train_2, X_test_2, y_test_2 = split_data(dataset_2)

In [23]:

compressed_dataset_X_2 = pcaTrain.transform(X_2) # --transform-- the original "dataset" Dataframe into a lower-dimensional space based on the model's learned parameters


compressed_dataset_X_train_2= pcaTrain.transform(X_train_2) # --transform-- the original "dataset" Dataframe into a lower-dimensional space based on the model's learned parameters
print(type(compressed_dataset_X_train_2))
print(len(compressed_dataset_X_train_2))


compressed_dataset_X_test_2= pcaTrain.transform(X_test_2) # --transform-- the original "dataset" Dataframe into a lower-dimensional space based on the model's learned parameters
print(type(compressed_dataset_X_test_2))
print(len(compressed_dataset_X_test_2))


<class 'numpy.ndarray'>
464716
<class 'numpy.ndarray'>
116296


# fit the XG-Boost

In [24]:
from sklearn.model_selection import KFold
K = 5 # 5 fold cross validation
kf = KFold(n_splits = K, random_state = 42, shuffle = True)

In [26]:
# Let us make predictions for each of the 5 models and find mean 
# of those predictions

from tqdm import tqdm
train = np.array(compressed_dataset_X_train_2)
target_train = np.array(y_train_2)

test = np.array(compressed_dataset_X_test_2)
# target_train = df_train['congestion'].values

test_preds = np.zeros((len(test)))
oof_preds = np.zeros((len(train)))
for train_index, val_index in tqdm(kf.split(compressed_dataset_X_train_2), total=5):
    train_X, valid_X = train[train_index], train[val_index]
    train_y, valid_y = target_train[train_index], target_train[val_index]
    xgb_params = study.best_trial.params
    model = xgb.XGBRegressor(**xgb_params)
    model.fit(train_X, train_y,eval_set=[(valid_X,valid_y)],early_stopping_rounds=100,verbose=False)
    # for train accuracy
    preds = model.predict(valid_X)
    oof_preds[val_index] = preds
    # for test accuracy
    preds = model.predict(test)
    test_preds += (preds)/5
print(np.abs(oof_preds-target_train).mean())

100%|██████████| 5/5 [11:01<00:00, 132.35s/it]

0.16747998661055868


In [27]:
target_test = np.array(y_test_2)

In [28]:
from sklearn.metrics import accuracy_score
# valid_Acc_SVC=accuracy_score(target_train,oof_preds)
# print(valid_Acc_SVC)

# another technique for calculate test accuracy
test_Acc_SVC=accuracy_score(target_test,test_preds.round())
print(test_Acc_SVC)

0.9206937469904382


In [29]:
len(oof_preds)

464716

In [31]:
valid_Acc=accuracy_score(target_train,oof_preds.round())
print(valid_Acc)

0.9164306802434175


In [ ]:
train_Acc=model.score(compressed_dataset_X_train_2,y_train_2)
print(train_Acc)

0.9721717086374351


In [ ]:
test_Acc=model.score(compressed_dataset_X_test_2,y_test_2)
print(test_Acc)

0.8712831805468825
